# Métodos de aprendizaje automático 2017 laboratorio 1

### En esta primera etapa se realizará un experimento para mostrar cómo se puede aplicar erroneamente la selección de características.

Relizamos la importación de los módulos necesarios.

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import *
from sklearn.metrics import *
from sklearn import feature_selection
import random
import numpy as np

Generamos una matriz con 10000 características aleatorias y una etiqueta a predecir también con valores aleatorios.

In [2]:
random.seed(0)
X = np.random.randint(0,10,size=[100,10000])
y = np.random.randint(0,2,size=100)

Aplicamos feature selection con el método de chi cuadrado.

In [3]:
fs = feature_selection.SelectKBest(feature_selection.chi2, k=20)
X_fs = fs.fit_transform(X, y)

Generamos un clasificador. En este caso una regresión logística.

In [4]:
clf = LogisticRegression()

Aplicamos validación cruzada para ver los resultados obtenidos.

In [5]:
scores = cross_val_score(clf, X_fs, y, cv=5, scoring='accuracy')
print scores

[ 0.95238095  0.85714286  0.8         0.89473684  0.78947368]


Explicar por qué si los números son generados al azar se obtienen buenos resultados. ¿Cómo solucionaría este problema?
Resolver el problema utilizando pipelines de sklearn

Respuesta : 
 Si se observa la siguiente línea: feature_selection.SelectKBest(feature_selection.chi2, k=20) . 
 Lo que hace esta línea es tomar los 20 atributos más representativos de X segun las target, utilizando el test
 de chi2, que calcula las dimensiones de X más independientes del target Y y las elimina. 
 Esta selección de atributos utilizando chi2 es apropiada para los casos de aprendizaje supervisado en problemas de regresión,
 es decir aquellos que tienen uno o mas valores en target que toman valores continuos, por ejemplo medir el largo
 de un animal a partir de su edad y peso. Los problemas donde los valores en target son discretos se llaman problemas
 de clasificación, este es nuestro caso ya que Y toma valores en 1..10. Para descartar que el problema venga por ese lado
 se hizo una prueba utilizando f_classif para select_k_best en vez de chi2.
 
 Prueba 1
 
 Resultado [ 0.95238095  0.9         0.9         0.9         0.94736842]

 Como vemos el resultado sigue siendo bueno (valores cercanos a 1).
 El problema entonces se encuentra en la seleccion de los k mejores atributos , se observa que utilizamos todo el conjunto X       para esta selección, luego en la validación, y por lo tanto los resultados son mejores que si se valida contra un conjunto de  ejemplos que no se haya utilizado en el entrenamiento ni en la selección de atributos, como se muestra en la siguiente prueba:
 
 Prueba 2
 
 Creamos el siguiente pipeline:
 Pipeline([('fs2', feature_selection.SelectKBest(feature_selection.chi2,k=20)), ('clf2',LogisticRegression()),]) 
 luego aplicamos validación cruzada utilizando este pipeline sobre el conjunto X.
 De esta forma para cada iteración se aplica primero select_k_best en base al conjunto entrenamiento y luego se valida
 contra el conjunto test, el cual se utiliza únicamente en esta instancia.
 
 Resultado [ 0.47619048  0.55        0.35        0.5         0.63157895]
 
 Observamos resultados maá razonables considerando que los datos son aleatorios. 

In [6]:
#Prueba 1
fs1 = feature_selection.SelectKBest(feature_selection.f_classif, k=20)
X_fs1 = fs1.fit_transform(X, y)
scores1 = cross_val_score(clf, X_fs1, y, cv=5, scoring='accuracy')
print scores1

#Prueba 2, resolucion ejercicio
from sklearn.pipeline import Pipeline
a2 = Pipeline([('fs2', feature_selection.SelectKBest(feature_selection.chi2,k=20)),('clf2',LogisticRegression()),])
a2 = a2.fit(X, y)
scores = cross_val_score(a2, X, y, cv=5, scoring='accuracy')
print scores

[ 0.80952381  0.85714286  1.          0.89473684  0.78947368]
[ 0.28571429  0.52380952  0.5         0.63157895  0.47368421]


### Para la siguiente parte trabajaremos con un conjunto de datos de textos. Para ello los descargamos.
Una descripción del set de datos: http://qwone.com/~jason/20Newsgroups/

In [7]:
from sklearn.datasets import fetch_20newsgroups
train = fetch_20newsgroups(subset='train', categories=['talk.politics.guns', 'soc.religion.christian', 'comp.graphics', 'sci.med'], shuffle=True, random_state=42)
test = fetch_20newsgroups(subset='test', categories=['talk.politics.guns', 'soc.religion.christian', 'comp.graphics', 'sci.med'], shuffle=True, random_state=42)

Mostrar un ejemplo de entrenamiento de cada una de las categorías.

In [8]:
for t in set(train.target):
    listcat = [index for index,value in enumerate(train.target) if value == t][:1]
    print(train.target_names[t])
    print train.data[listcat[0]]

comp.graphics
Subject: newss
From: pollarda@physc1.byu.edu
Distribution: world
Organization: Brigham Young University
Lines: 24



I am working on a project where we are going to be including
both still and moving grapics within a database.  Of course
JPEG and MPEG come to mind as the formats of choice for the
various files.  However, from what I read on the Net, it seems
as if there are several different forms of each of these.

What I want to do, is settle on a file format which I can count
on as being a standard format 10 years from now.  I know Apple is going
to support Quicktime on the new Power PC's and, so this
may be the format of choice.

What format does Apple's Quicktime use for their products?  I guess
it is some kind of MPEG for their motion picture. Is it any different
than standard MPEG files?

Thanx for any info!

Art.
Pollarda@xray.byu.edu


  

sci.med
From: annick@cortex.physiol.su.oz.au (Annick Ansselin)
Subject: Re: Is MSG sensitivity superstition?
Nntp-Posting-Hos

Revisar el módulo CountVectorizer para extraer caracterísitcas a partir de texto y explicar cómo funciona. Explicar qué es el modelo de bolsa de palabras.

Respuesta : 

 CountVectorizer es una librería para procesar textos y poder clasificarlos. Convierte una colección de documentos en una matriz de conteo de ocurrencias de palabras.
 Supongamos que se tienen N documentos cada uno de ellos pertenecientes a una categoria t (target) que se quiere predecir.
 CountVectorizer crea una matriz de tamaño N x cantF, donde cantF es el conjunto de palabras que aparecen en algún 
 documento, estas son las features consideradas para el entrenamiento.
 Cada documento se considera una bolsa de palabras, es decir un conjunto de palabras sin importar su orden ni dependencia en el documento, y CV cuenta la cantidad de ocurrencias de cada palabra j para cada documento i y setea el valor  (i,j)=cantOcurrencias en la matriz. 
 Al proceso de asignar un numero en el rango de [1..cantF] para cada palabra se le llama tokenización, y al de contar las  ocurrencias se le llama conteo.
 De esta forma, aplicando fit_transform ya tenemos un conjunto de entrenamiento dado por la matriz creada y podemos utilizarlo 
 para entrenar, sin embargo hay un par de ajustes que pueden mejorar la performance. Uno de ellos es la cantidad de valores 0 
 que tiene nuestra matriz, supongamos que tenemos 100.000 features(conjunto de palabras en todos los documentos) y un documento 
 pequeño con 200 palabras. Entonces en la row de ese documento vamos a tener como mínimo 99.800 valores en cero. La librería 
 scipy.sparse soluciona este problema almacenando únicamente valores distintos de cero y ahorrando así mucha memoria.
 
 Otro de los procesos que se puede aplicar es el de normalización (explciado en la parte 10 del ejercicio). En resumen, "bolsa de palabras" es el proceso que sigue los pasos tokenización, conteo y normalizacion. Considerando únicamente ocurrencias de palabras e ignorando la posición relativa de las mismas.

In [9]:

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train.data)


Entrenar un clasificador bayesiano sencillo Multinomial. Explicar cada uno de los parámetros de este clasificador.

Respuesta:

MultinomialNB es una implementación  del algoritmo naive Bayes para datos con distribución multinomial. La distribución es parametrizada por el vector ![](img/1.png)  para cada clase de Y, donde n es la cantidad de features y 0_yi es la probabilidad ![](img/2.png)  de que aparezca el feature  en una muestra perteneciente a la clase .
Cada parámetro 0_y se calcula con una versión suavizada  de conteo de frecuencia relativa:![](img/3.png)
con ![](img/4.png)  la cantidad de veces que aparece el feature i en la muestra de la clase y en el conjunto de entrenamiento T, y ![](img/5.png) es la cuenta total para todos los features en la muestra de la clase .

El parámetro ∝ sirve para el suavizado de la función  y sirve para que la probabilidad de los features que no salieron en la muestra sea distinta de 0, si se usa ∝ =0 la función no tiene suavizado. Si ∝ =1 se le llama suavizado de Laplace, y si ∝ >1 se le llama suavizado de Lidstone.
Los parámetros fit_prior y class_prior sirven para definir las probabilidades preferentes de las clases. Class_prior puede ser un arreglo del tamaño del conjunto de clases Y, o puede ser None. Si se pasa un arreglo, éste lleva los valores de probabilidades de cada clase. En caso de que el valor sea None las probabilidades de las clases se calculan en función de fit_prior. Si fit_prior es True se calcula las probabilidades en función de los datos, si es False se usa una probabilidad uniforme para todas las clases. 


In [10]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True).fit(X_train_counts, train.target)


Repetir el proceso pero utilizando el módulo TfidfTransformer en lugar de CountVectorizer. Explicar la medida Tfidf.

Respuesta:

 Como se explicó en la parte de CountVectorizer, la matriz creada tiene algunos inconvenientes que pueden
 afectar la performance del clasificador. Supongamos un documento muy grande que pertenece a la misma categoría
 que un documento pequeño, la cantidad de ocurrencias de una palabra en común para el documento grande es mayor 
 a la del documento chico, cuando en realidad estan representando lo mismo pero a distinta escala. Para solucionar esto simplemente se divide la cantidad de ocurrencias de cada palabra en un documento
 por la cantidad de palabras en el mismo, a estas nuevas features se les llama "Term Frequencies".

 Otra mejora esta en considerar menos relevantes palabras que aparecen repetidas en el cuerpo de varios documentos
 que aquellas que aparecen en una porción pequeña del cuerpo del documento, se ajustan los pesos relativos a cada palabra 
 para solucionarlo. A esta técnica se le llama "Term Frequency times Inverse Document Frequency", y la combinación de ambas
 se puede aplicar utilizando TfidfTransformer, como se muestra en la siguiente prueba.
 

In [11]:


from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clfb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True).fit(X_train_tfidf, train.target)

Incluya los pasos de extracción de características y el clasificador dentro de un pipeline.

In [12]:
pipeline = Pipeline([('count_vect', CountVectorizer()),('tfidf_transformer',TfidfTransformer()),('clf',MultinomialNB()),])
pipeline.fit(train.data,train.target)

Pipeline(steps=[('count_vect', CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
    ...False,
         use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

Incluir las métricas de precision, recall y fscore para cada cada una de las clases y la matriz de confusión probando contra el conjunto de testing.

Aclación: Se agregan las medidas solicitadas, pero se utiliza el parametro "average=macro".

In [13]:
from sklearn.metrics import precision_recall_fscore_support


predicted = pipeline.predict(test.data)
res =  precision_recall_fscore_support(test.target, predicted, average='macro')
print "Precision: %f " % res[0],
print " Recall: %f " % res[1],
print " F-score: %f " % res[2]

print "Confusion matrix"
print confusion_matrix(test.target, predicted)

(array([ 0.96368715,  0.98765432,  0.80284553,  0.9383378 ]), array([ 0.88688946,  0.80808081,  0.99246231,  0.96153846]), array([ 0.92369478,  0.88888889,  0.88764045,  0.94979647]), array([389, 396, 398, 364]))
[[345   2  29  13]
 [  9 320  57  10]
 [  2   1 395   0]
 [  2   1  11 350]]


Incluya una GridSearch para ver cuales son los mejores hiper-parámetros del modelo.

Aclación: No se agregan todos los parámetros posibles 

In [22]:
from sklearn.model_selection import GridSearchCV

tuned_parameters = {  
   'count_vect__max_df': (0.5, 1.0),  
   'count_vect__max_features': (10000, 20000),  
   'count_vect__min_df': (1, 5),  
   'tfidf_transformer__use_idf': (True, False),  
   'tfidf_transformer__sublinear_tf': (True, False),  
   'tfidf_transformer__norm': ('l1', 'l2'),  
   'clf__alpha': (0.1, 0.01)  
   }
grid_search = GridSearchCV(pipeline, tuned_parameters, cv=3,scoring='accuracy',n_jobs = 2)
grid_search.fit(train.data, train.target)
best_parameters = grid_search.best_estimator_.get_params()

print 'Mejores hiper-parametros encontraodos'
for param_name in sorted(list(tuned_parameters.keys())): print("\t{0}: {1}".format(param_name, best_parameters[param_name]))

Mejores hiper-parametros encontraodos
	clf__alpha: 0.01
	count_vect__max_df: 0.5
	count_vect__max_features: 20000
	count_vect__min_df: 1
	tfidf_transformer__norm: l2
	tfidf_transformer__sublinear_tf: True
	tfidf_transformer__use_idf: False


Explique cada uno de los parámetros de cada uno de los pasos del pipeline y pruebe con varios tipos de clasificadores de los vistos en el curso. ¿Con qué configuración de parámetros obtuvo mejores resultados? Aplique selección de atributos y valore los resultados obtenidos. Realice una prueba final contra el conjunto de pruebas.

Respuesta:

Los parámetros que se modificaron en la grid search para cada paso del pipelines son: max_df,mi_df y max_features para el CountVectorizer, use_idf, sublinear_tf y norm para TfidfTranformer y alpha para multinomialNB.

max_df : float in range [0.0, 1.0] or int, default=1.0
Al armar el vocabulario de palabras identificadas por countVectorizer, max_df es la frecuencia máxima que puede tener una palabra, no se tienen en cuenta las palabras que aparecen con una frecuencia superior.Se puede pasar un float entre 0 y 1 inclusive o un entero, si se pasa un valor de tipo float el límite se toma como la fecuencia de aparición de la palabra en documentos, si se pasa un valor entero el límite se toma como la cantidad absoluta de apariciones de la palabra en todos los documentos.
En el gridsearch se probó con los valores 0.5 y 1.0, que significan respecivamente,eliminar las palabras que aparecen en la mitad o más de los documentos, y eliminar las palabras que aparecen en absoutamente todos los documentos.

min_df : float in range [0.0, 1.0] or int, default=1
Este parámetro es el análogo del anterior pero para el mínimo. No tiene en cuenta para el vocabulario las palabras que aparecen con frecuencia inferior a la marcada. Cumple la misma lógica de aceptar un valor de tipo float entre o y 1 o un entero positivo.
Se probaron los valores 1 y 5, es decir que no se tengan en cuenta los términos que apaerecen menos de 1 y 5 veces respectivamente.

max_features : int or None, default=None
Si se utiliza un valor entero para este parámetro entonces el vocabulario considera solamente los primeros n términos con mayor frecuencia,siendo n el entero ingresado como max_features. Si se utiliza el valor None entonces se consideran todos los términos del vocabukario encontrado.
Se usaron los valores 1000 y 2000 para probar.

Estos tres parámetros son ignorados si countVectorizer toma un vocabulario inicial, ya que en ese caso el vocabulario ya esta definido y countVectorizer no arma uno propio,sino que considera solamente las palabras del vocabulario.

norm : ‘l1’, ‘l2’ or None, optional
Se elige que función se usa para normalizar los valores.En caso de usar el valor None no se hace normalización.
En el gridsearch realizado se probó con las normalizaciones de tipo l1 y l2.

use_idf : boolean, default=True
Sirve para activar el balanceo por frecuencia inversa. Es decir que si se pasa el valor True(esto se hace por defecto) se usa la medida Tfidf , mientras que si se pasa el valor False se usa solo la medida Tf(Term frequency).
Se realizó la prueba con el los valores True y False,es decir que se probó con las medidas TfidF y Tf respectivamente.

sublinear_tf : boolean, default=False
Define si se aplica ajuste sublineal a Tf.Si el valor es True se reemplaza tf con 1 + log(tf).
Se realizó la prueba con los valores True y False.

alpha : float, optional (default=1.0)
Este parámetro como fue explicado en la pregunta sobre multinomialNB , se usa para el suavizado del cálculo de probabilidades empíricas. Sirve para que los términos que no aparecen en los casos de entrenamiento puedan tener una probabilidad por más que no se hayan encontrado ocurrencias en los documentos usados para entrenar.
Para probar con gridSearch se usaron los valores 0.1 y 0.001

Los parámetros que mejor resultado dieron se muestran en el resultado del cñodigo de la pregunta anterior.

In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

names = ['KNeighbors Classifier','MultinomialNB Classifier','DecisionTree Classifier','Multi-layer Perceptron Classifier']
models = list()

pipeline_KN = Pipeline([('count_vect', CountVectorizer(max_df = best_parameters['count_vect__max_df'],max_features=best_parameters['count_vect__max_features'],min_df=best_parameters['count_vect__min_df']))
                        ,('tfidf_transformer',TfidfTransformer(norm=best_parameters['tfidf_transformer__norm'],sublinear_tf=['tfidf_transformer__sublinear_tf'],use_idf=best_parameters['tfidf_transformer__use_idf'])),
                        ('fs',SelectKBest(chi2, k=15000)),('clf',KNeighborsClassifier()),])
models.append(pipeline_KN)
pipeline_MNB = Pipeline([('count_vect', CountVectorizer(max_df = best_parameters['count_vect__max_df'],max_features=best_parameters['count_vect__max_features'],min_df=best_parameters['count_vect__min_df']))
                        ,('tfidf_transformer',TfidfTransformer(norm=best_parameters['tfidf_transformer__norm'],sublinear_tf=['tfidf_transformer__sublinear_tf'],use_idf=best_parameters['tfidf_transformer__use_idf'])),
                        ('fs',SelectKBest(chi2, k=15000)),('clf',MultinomialNB()),])
models.append(pipeline_MNB)
pipeline_DTree= Pipeline([('count_vect', CountVectorizer(max_df = best_parameters['count_vect__max_df'],max_features=best_parameters['count_vect__max_features'],min_df=best_parameters['count_vect__min_df']))
                        ,('tfidf_transformer',TfidfTransformer(norm=best_parameters['tfidf_transformer__norm'],sublinear_tf=['tfidf_transformer__sublinear_tf'],use_idf=best_parameters['tfidf_transformer__use_idf'])),
                        ('fs',SelectKBest(chi2, k=15000)),('clf',DecisionTreeClassifier()),])
models.append(pipeline_DTree)
pipeline_MLPC= Pipeline([('count_vect', CountVectorizer(max_df = best_parameters['count_vect__max_df'],max_features=best_parameters['count_vect__max_features'],min_df=best_parameters['count_vect__min_df']))
                        ,('tfidf_transformer',TfidfTransformer(norm=best_parameters['tfidf_transformer__norm'],sublinear_tf=['tfidf_transformer__sublinear_tf'],use_idf=best_parameters['tfidf_transformer__use_idf'])),
                        ('fs',SelectKBest(chi2, k=15000)),('clf',MLPClassifier()),])
models.append(pipeline_MLPC)

max_score = {'BestClassifier_name':'','BestClassifier_score':0,'BestClassifier_model':None}
## Prueba mejor clasificador##
for model,name in zip(models,names):   
    print '\nResultado promedio para %s' % (name)
    score = np.mean(cross_val_score(model, train.data, train.target, cv=5, scoring='accuracy'))  
    print score
    if score > max_score['BestClassifier_score']:
        max_score['BestClassifier_name'] = name
        max_score['BestClassifier_score'] = score
        max_score['BestClassifier_model'] = model
        
print '\nMejor clasificador: %s Score: %f \n' % (max_score['BestClassifier_name'],max_score['BestClassifier_score']) 

## Prueba contra conjunto de testing ##
print "Evaluacion contra conjunto de pruebas"
best_model = max_score['BestClassifier_model']
best_model.fit(train.data,train.target)
predicted = best_model.predict(test.data)
res =  precision_recall_fscore_support(test.target, predicted, average='macro')
print "Precision: %f " % res[0],
print " Recall: %f " % res[1],
print " F-score: %f " % res[2]


Resultado promedio para KNeighbors Classifier
0.927688606562

Resultado promedio para MultinomialNB Classifier
0.956093479643

Resultado promedio para DecisionTree Classifier
0.820443520668

Resultado promedio para Multi-layer Perceptron Classifier
0.980625563001

Mejor clasificador: Multi-layer Perceptron Classifier Score: 0.980626 

Evaluacion contra conjunto de pruebas
Precision: 0.955958   Recall: 0.954946   F-score: 0.954942 


Repita el ejercicio utilizando HashingVectorizer para extraer características. Explique las ventajas de este método de extracción de caraterísticas frente a CountVectorizer.

Respuesta:

Como se explico anteriormente, CountVectorizer considera los documentos como una bolsa de palabras, tomando en cuenta 
únicamente la cantidad de ocurrencias. Esta estrategia tiene dos problemas, el primero es que al considerar tokens como palabras individuales no estamos considerando frases y expresiones (formadas por varias palabras) en los documentos, esta información puede aportar para la clasificación si se logra extraerla. 
El segundo problema que presenta es la incapacidad de detectar faltas ortográficas, como ejemplo supongamos dos documentos con cuerpos "arbol" y "abrol", el segundo tiene una falta de ortografía, pero CountVectorizer las considera palabras distintas. Para solucionarlo una estrategia puede ser considerar tokens formados por caracteres de largo n (incluyendo los espacios), de esta forma el resultado de tokenizar nuestro ejemplo sería ['ar','rb','bo','ol','ab','br','ro'], teniendo las dos palabras en comun por lo menos algunas features. A este resultado se le llama "n-grams collection" mientras que la bolsa de palabras es una "unigram collection".
La aplicacion descrita en el parrafo anterior es simple pero tiene algunos problemas sobre todo de performance y uso de memoria.
La libreria HashingVectorizer los soluciona, la diferecnia entre CountVectorizer y HashingVectorizer es la forma en la que Hashingvectorizer ¨tokeniza¨ las palabras. CountVectorizer calcula la cantidad de ocurrencias para cada palabra. HashingVectorizer lo que hace es usar una función de Hashing para asociar de manera veloz un número a determinada palabra del documento y de esta forma aplica las etapas de ¨tokenizar¨ y contar en una misma etapa.

En resumen, HashingVectorizer combina las tecnicas Hashing y CountVectorizer para mejorar los resultados.

Ventajas de HashingVectorizer:
-Del punto de vista de memoria es muy escalable para datasets muy grandes,ya que no tiene necesidad de almacenar un diccionario del vocabulario en memoria.
-Es rápido para serialización y deserialización ya que no tiene una estado asociado.
Puede ser usado en un streaming (partial fit) o parallel pipeline ya que no se computa ningún estado durante el fit.

Desventajas:
-Si bien podemos transformar un documento en un conjunto de tokens y luego en indices de las features, no es posible hacer el proceso inverso, esto puede dar problema a la hora de analizar los resultados por ejemplo para ver que features son mas relevantes para un grupo.
-Puede haber colisiones y almacenarse dos tokens distintos dentro de un mismo indice de feature.
-No aplica IDF weighting como lo haria un transformador con estado.

In [17]:
from sklearn.feature_extraction.text import HashingVectorizer
pipeline_MNB_ = Pipeline([('hash_vect', HashingVectorizer(non_negative=True)),('tfidf_transformer',TfidfTransformer()),('clf',MultinomialNB()),])

print "Resultado cross-validation con HashingVectorizer para el MultinomialNB"
print cross_val_score(pipeline_MNB_, train.data, train.target, cv=5, scoring='accuracy')

tuned_parameters = {
    'hash_vect__ngram_range': ((2,2),(1,1)),
    'hash_vect__n_features':(2**18,2**19,2**20),
    'hash_vect__lowercase':(True,False),
    'hash_vect__norm':(None,'l1')  
   }
grid_search = GridSearchCV(pipeline_MNB_, tuned_parameters, cv=3,scoring='accuracy',n_jobs = 2)
grid_search.fit(train.data, train.target)
best_parameters = grid_search.best_estimator_.get_params()

print '\nMejores hiper-parametros encontraodos para el HashingVectorizer'
for param_name in sorted(list(tuned_parameters.keys())): print("\t{0}: {1}".format(param_name, best_parameters[param_name]))

Resultado cross-validation con HashingVectorizer para el MultinomialNB
[ 0.92060086  0.92903226  0.95268817  0.95913978  0.95238095]

Mejores hiper-parametros encontraodos para el HashingVectorizer
	hash_vect__lowercase: True
	hash_vect__n_features: 262144
	hash_vect__ngram_range: (1, 1)
	hash_vect__norm: None
